In [1]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder

from sklearn.preprocessing import MinMaxScaler
from sklearn.feature_selection import SelectKBest, chi2, RFE, SelectFromModel
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from lightgbm import LGBMClassifier

import matplotlib.pyplot as plot

In [2]:
df = pd.read_csv('Hotel_Reservations.csv')

In [3]:
df.head()

,Booking_ID,no_of_adults,no_of_children,no_of_weekend_nights,no_of_week_nights,type_of_meal_plan,required_car_parking_space,room_type_reserved,lead_time,arrival_year,arrival_month,arrival_date,market_segment_type,repeated_guest,no_of_previous_cancellations,no_of_previous_bookings_not_canceled,avg_price_per_room,no_of_special_requests,booking_status
0,INN00001,2,0,1,2,Meal Plan 1,0,Room_Type 1,224,2017,10,2,Offline,0,0,0,65.00,0,Not_Canceled
1,INN00002,2,0,2,3,Not Selected,0,Room_Type 1,5,2018,11,6,Online,0,0,0,106.68,1,Not_Canceled
2,INN00003,1,0,2,1,Meal Plan 1,0,Room_Type 1,1,2018,2,28,Online,0,0,0,60.00,0,Canceled
3,INN00004,2,0,0,2,Meal Plan 1,0,Room_Type 1,211,2018,5,20,Online,0,0,0,100.00,0,Canceled
4,INN00005,2,0,1,1,Not Selected,0,Room_Type 1,48,2018,4,11,Online,0,0,0,94.50,0,Canceled


---
### Exploratory Data Analysis 

In [4]:
df.shape

(36275, 19)

##### Observation: We have too many features and few samples, taking into account the number of features.

In [5]:
df.columns

Index(['Booking_ID', 'no_of_adults', 'no_of_children', 'no_of_weekend_nights',
       'no_of_week_nights', 'type_of_meal_plan', 'required_car_parking_space',
       'room_type_reserved', 'lead_time', 'arrival_year', 'arrival_month',
       'arrival_date', 'market_segment_type', 'repeated_guest',
       'no_of_previous_cancellations', 'no_of_previous_bookings_not_canceled',
       'avg_price_per_room', 'no_of_special_requests', 'booking_status'],
      dtype='object')

In [6]:
df.describe().transpose()

,count,mean,std,min,25%,50%,75%,max
no_of_adults,36275.0,1.844962,0.518715,0.0,2.0,2.00,2.0,4.0
no_of_children,36275.0,0.105279,0.402648,0.0,0.0,0.00,0.0,10.0
no_of_weekend_nights,36275.0,0.810724,0.870644,0.0,0.0,1.00,2.0,7.0
no_of_week_nights,36275.0,2.204300,1.410905,0.0,1.0,2.00,3.0,17.0
required_car_parking_space,36275.0,0.030986,0.173281,0.0,0.0,0.00,0.0,1.0
lead_time,36275.0,85.232557,85.930817,0.0,17.0,57.00,126.0,443.0
arrival_year,36275.0,2017.820427,0.383836,2017.0,2018.0,2018.00,2018.0,2018.0
arrival_month,36275.0,7.423653,3.069894,1.0,5.0,8.00,10.0,12.0
arrival_date,36275.0,15.596995,8.740447,1.0,8.0,16.00,23.0,31.0
repeated_guest,36275.0,0.025637,0.158053,0.0,0.0,0.00,0.0,1.0


In [7]:
df.isnull().sum()

Booking_ID                              0
no_of_adults                            0
no_of_children                          0
no_of_weekend_nights                    0
no_of_week_nights                       0
type_of_meal_plan                       0
required_car_parking_space              0
room_type_reserved                      0
lead_time                               0
arrival_year                            0
arrival_month                           0
arrival_date                            0
market_segment_type                     0
repeated_guest                          0
no_of_previous_cancellations            0
no_of_previous_bookings_not_canceled    0
avg_price_per_room                      0
no_of_special_requests                  0
booking_status                          0
dtype: int64

### Observations:
The items below were dropped from the dataset, since they didn't increase the accuracy of the model, moreover, they were categorical items. 
- room_type_reserved
- type_of_meal_plan
- Booking_ID

In [8]:
df['market_segment_type'] = LabelEncoder().fit_transform(df['market_segment_type'])

In [9]:
# df.drop(['Booking_ID', 'type_of_meal_plan', 'room_type_reserved'], axis=1, inplace=True)
# df.drop(['Booking_ID', 'type_of_meal_plan'], axis=1, inplace=True) no parameters accuracy score 0.7754295690526509 and 0.7583386933749885 with huber
# df.drop(['Booking_ID', 'room_type_reserved'], axis=1, inplace=True) no parameters accuracy score 0.67 and 0.7746025911972801 with huber
# df.drop(['Booking_ID'], axis=1, inplace=True) no parameters accuracy score 0.7665165855003216 and 0.771110906919048 with huber
# df.drop(['Booking_ID'], axis=1, inplace=True)
# df.drop(['Booking_ID', 'type_of_meal_plan', 'room_type_reserved'], axis=1, inplace=True) no parameters accuracy score 0.7723973169162914 and 0.7655058347882018 with huber
df.drop(['Booking_ID', 'type_of_meal_plan', 'room_type_reserved'], axis=1, inplace=True)

---
### Training the model 

In [10]:
from sklearn.model_selection import train_test_split

In [11]:
X = df.drop('booking_status', axis=1)
y = df['booking_status']

In [12]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=1)

In [13]:
from sklearn.linear_model import SGDClassifier
from sklearn.metrics import confusion_matrix, accuracy_score

In [14]:
SGD_model = SGDClassifier()

---
### Fitting the model

In [15]:
SGD_model.fit(X_train, y_train)

SGDClassifier()

##### When fitting the model, after traning it without the followings parameters: max_iter=1000, tol=1e-3. It displayed a ValueError, and the root cause is:  
    704     warnings.warn(
    705         "Maximum number of iteration reached before "
    706         "convergence. Consider increasing max_iter to "
    707         "improve the fit.",
    708         ConvergenceWarning,
    709     )

In [16]:
SGD_predictions = SGD_model.predict(X_test)

In [17]:
confusion_matrix(y_test, SGD_predictions)

array([[1356, 2166],
       [ 378, 6983]])

In [18]:
accuracy_score(y_test, SGD_predictions)

0.766240926215198

In [19]:
accuracy_score(y_test, SGD_predictions, normalize=False)

8339

##### Testing with different parameters 

In [20]:
SGD_model2 = SGDClassifier(loss='modified_huber', alpha=0.001)

In [21]:
SGD_model2.fit(X_train, y_train)

SGDClassifier(alpha=0.001, loss='modified_huber')

In [22]:
SGD_predictions2 = SGD_model2.predict(X_test)

In [23]:
confusion_matrix(y_test, SGD_predictions2)

array([[ 624, 2898],
       [  88, 7273]])

In [24]:
accuracy_score(y_test, SGD_predictions2)

0.7256271248736562

In [25]:
accuracy_score(y_test, SGD_predictions2, normalize=False)

7897

---

In [26]:
# loss='log_loss' alpha=0.0001
SGD_model3 = SGDClassifier(loss='log_loss', alpha=0.0001)

In [27]:
SGD_model3.fit(X_train, y_train)

SGDClassifier(loss='log_loss')

In [28]:
SGD_predictions3 = SGD_model3.predict(X_test)

In [29]:
confusion_matrix(y_test, SGD_predictions3)

array([[3173,  349],
       [3847, 3514]])

In [30]:
accuracy_score(y_test, SGD_predictions3)

0.614444546540476